In [4]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

def load_nifti_image(file_path):
    """
    Carica un'immagine NIfTI (.nii.gz) e restituisce i dati dell'immagine come array numpy.
    """
    img = nib.load(file_path)
    img_data = img.get_fdata()
    return img_data

def get_slice_with_largest_label(mri_path, label_path):
    """
    Carica l'immagine MRI e le label, e restituisce lo slice con la label più grande.
    """
    mri_data = load_nifti_image(mri_path)
    label_data = load_nifti_image(label_path)
    # Rendi tutti i valori != 0 uguali a 1
    label_data[label_data != 0] = 1
    # Trova lo slice con la label più grande
    slice_index = np.argmax(np.sum(label_data, axis=(0, 1)))
    
    # Restituisce lo slice dell'immagine MRI corrispondente
    mri_slice = mri_data[:, :, slice_index]
    label_slice = label_data[:, :, slice_index]
    
    return mri_slice, label_slice
def visualize_slice(mri_slice, label_slice, mri_title='MRI', label_title='Label'):
    """
    Visualizza lo slice dell'immagine MRI e delle label.
    """
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(mri_slice, cmap='gray')
    ax[0].set_title(mri_title)
    ax[1].imshow(label_slice, cmap='Reds', vmin=0, vmax=1)
    ax[1].set_title(label_title)
    plt.show()
def dice_score(mask1, mask2):
    """
    Calcola il coefficiente di Dice tra due maschere binarie.
    """
    intersection = np.sum(mask1 * mask2)
    return 2. * intersection / (np.sum(mask1) + np.sum(mask2))


In [5]:
mri_folder_path = '..\\..\\Task03_Liver\\Task03_Liver\\imagesTr\\'
label_folder_path = '..\\..\\Task03_Liver\\Task03_Liver\\labelsTr\\'

In [7]:
import segmentation_functions as sf
dice_scores = {
    'watershed': [],
    'exploration': [],
    'flood': []
}
N=10
for i in range(0,N+1):

    mri_path = mri_folder_path+f'liver_{i}.nii.gz'
    label_path = label_folder_path + f'liver_{i}.nii.gz'
    mri_slice, label_slice = get_slice_with_largest_label(mri_path, label_path)
    wshedmask=sf.watershed_segmentation(mri_slice)
    exploremask=sf.exploration_segmentation(mri_slice,label_slice)
    floodmask=sf.flood_segmentation(mri_slice,label_slice)
    #visualize_slice(mri_slice, label_slice)
    #visualize_slice(mri_slice, wshedmask, mri_title='MRI', label_title='Watershed mask')
    #visualize_slice(mri_slice, exploremask, mri_title='MRI', label_title='Exploration mask')
    #visualize_slice(mri_slice, floodmask, mri_title='MRI', label_title='Flooded mask')

    dice_scores['watershed'].append(dice_score(wshedmask, label_slice))
    dice_scores['exploration'].append(dice_score(exploremask, label_slice))
    dice_scores['flood'].append(dice_score(floodmask, label_slice))
    """print(f"Iteration {i}:")
    print(f"Dice score (Watershed): {dice_scores['watershed'][-1]}")
    print(f"Dice score (Exploration): {dice_scores['exploration'][-1]}")
    print(f"Dice score (Flood): {dice_scores['flood'][-1]}")

    print(dice_scores)"""

Iteration 0:
Dice score (Watershed): 0.9237512584217455
Dice score (Exploration): 0.9619152876515921
Dice score (Flood): 0.909079365079365
{'watershed': [0.9237512584217455], 'exploration': [0.9619152876515921], 'flood': [0.909079365079365]}
Iteration 1:
Dice score (Watershed): 0.9628600745801736
Dice score (Exploration): 0.974172390322381
Dice score (Flood): 0.9321181320818316
{'watershed': [0.9237512584217455, 0.9628600745801736], 'exploration': [0.9619152876515921, 0.974172390322381], 'flood': [0.909079365079365, 0.9321181320818316]}
Iteration 2:
Dice score (Watershed): 0.2824533011852425
Dice score (Exploration): 0.3928963681379028
Dice score (Flood): 0.09740274069333804
{'watershed': [0.9237512584217455, 0.9628600745801736, 0.2824533011852425], 'exploration': [0.9619152876515921, 0.974172390322381, 0.3928963681379028], 'flood': [0.909079365079365, 0.9321181320818316, 0.09740274069333804]}
Iteration 3:
Dice score (Watershed): 0.0
Dice score (Exploration): 0.9053594434424117
Dice sc

In [8]:
print(f"Average Dice score (Watershed): {np.mean(dice_scores['watershed'])}")
print(f"Average Dice score (Exploration): {np.mean(dice_scores['exploration'])}")
print(f"Average Dice score (Flood): {np.mean(dice_scores['flood'])}")

Average Dice score (Watershed): 0.296585998519973
Average Dice score (Exploration): 0.41940058756738047
Average Dice score (Flood): 0.2860054446834167
